Installation is simple:

```
Pkg.add("Stan")
```

You need to install CmdStan first though. Get it from [here](http://mc-stan.org/users/interfaces/cmdstan). You also need to set the `CMDSTAN_HOME` environment variable, which should point to the folder in which you unpacked `CmdStan`.

This notebook assumes that you have created a `CmdStanDir` variable (mostly likely by setting it in your `~/.juliarc.jl` file) and that it points to the location in which you unpacked CmdStan.

In [ ]:
using Stan

# Use the describe function from Mamba.
#
using Mamba

Check that environment variable is set.

In [ ]:
ENV["CMDSTAN_HOME"]

## Load Data

In [ ]:
using RData

In [ ]:
pwd()

In [ ]:
slots = load("/home/colliera/proj/Z-317-talk-first-steps-stan/maud-sessions.rda", convert=true)
slots = slots["sessions_stats"];

In [ ]:
size(slots, 1)

In [ ]:
names(slots)

In [ ]:
slots[1:6,:]

Define a model.

In [ ]:
const binomial_model = "
data {
  int<lower=0> N;
  int hits[N];
  int spins[N];
}
parameters {
  real<lower=0,upper=1> theta;
}
model {
  hits ~ binomial(spins, theta);       // Likelihood
  theta ~ beta(2, 2);                  // Prior
}
";

Create a `Stanmodel` object. This specifies the basic attributes for the model.

In [ ]:
stanmodel = Stanmodel(model=binomial_model, nchains=4, num_samples=1000, num_warmup=1000);

That will actually take the model string and write it to a temporary external file with `.stan` extension. The `name` argument can be used to specify the name of this file.

Take a look at the model object.

In [ ]:
stanmodel |> display

Interrogate attributes of the model.

In [ ]:
stanmodel.method

Update characteristics of model. The `Stanmodel` object can be tailored either via the constructor or after the fact.

In [ ]:
stanmodel.method.adapt.delta = 0.85

Set up the observed input data.

In [ ]:
const binomial_data = Dict("N"     => size(slots, 1),
                           "hits"  => slots[:hits],
                           "spins" => slots[:spins])

Build the code and run the simulation. Uses `make` to recompile code, but only if necessary. Multiple chains are run in parallel (if possible).

In [ ]:
rc, simulated = stan(stanmodel, [binomial_data], diagnostics=false);

Check on whether the simulation completed without error.

In [ ]:
rc

In [ ]:
stanmodel.command

Subset sampler output.

In [ ]:
monitor = ["theta", "lp__"]
#
simulated = simulated[:, monitor, :]

## Diagnostics

`Mamba` provides a range of [convergence diagnostics](http://mambajl.readthedocs.io/en/latest/tutorial.html#convergence-diagnostics).

In [ ]:
gelmandiag(simulated, mpsrf=true, transform=true) |> showall

The diagnostic of Geweke tests for non-convergence of posterior mean estimates per chain. Convergence is rejected for significant p-values.

In [ ]:
println("Geweke Convergence Diagnostic (one block for each chain)")
gewekediag(simulated) |> display

In [ ]:
heideldiag(simulated) |> showall

## Interrogating the Chains

Check for success and examine results. The `simulated` object is of type `Mamba.Chains`.

In [ ]:
describe(simulated)

In [ ]:
describe(simulated[:, ["theta"], :], q=[0.025, 0.5, 0.975])

Highest Posterior Density Intervals.

In [ ]:
hpd(simulated)

Cross-Correlations.

In [ ]:
cor(simulated)

Lag-Autocorrelations.

In [ ]:
autocor(simulated)

In [ ]:
dark_panel = Theme(
    panel_fill="black",
    default_color="orange"
)
Gadfly.push_theme(dark_panel)
p = plot(simulated[:, ["theta"], :], [:trace, :mean, :density, :autocor], legend=false);
draw(p)
# draw(p, ncol=2, filename="summaryplot", fmt=:svg)
draw(p, ncol=2, filename="summaryplot.pdf", fmt=:pdf, width=8inch, height=4inch)

In [ ]:
## Default summary plot (trace and density plots)
p = plot(simulated)

## Write plot to file
# draw(p, filename="summaryplot.svg")
draw(p)